## Pre-processing high street into vacancy survival format

In [1]:
# import necessary libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from requests import get
from urllib.parse import urlparse
from datetime import datetime

In [3]:
os.getcwd()

'/home/jovyan/work/Dissertation'

### Read in the high street - testing with Brecknock

In [53]:
Brecknock = pd.read_csv("Camden High Streets/Brecknock-points.csv", low_memory=False)

In [54]:
Brecknock = Brecknock.sort_values(by= ['status_date'])

In [55]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [56]:
#deleted those cols columns from the dataset as they only contain nulls

Brecknock = Brecknock.drop(cols, axis=1)

In [57]:
Brecknock['new_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [58]:
Brecknock = Brecknock.sort_values(by= ['new_date'])

In [59]:
Brecknock['new_date'].loc[(Brecknock['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [60]:
Brecknock = Brecknock.sort_values(by= ['location_code', 'period'])

In [61]:
Brecknock['locaton_code'] = Brecknock['location_code'].astype("category")

In [62]:
locations = Brecknock['location_code'].unique()

In [63]:
Brecknock.loc[Brecknock['name'].isnull(), 'name'] = Brecknock['location_code']

In [64]:
Brecknock['status_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [65]:
Brecknock['status_date'].iloc[Brecknock['status_date'] < '2010-04-01'] = '04/01/2010'

In [66]:
Brecknock['status_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [67]:
Brecknock['period2'] = pd.to_datetime(Brecknock['period'])

In [68]:
Brecknock['period2'] = Brecknock['period2'] + pd.DateOffset(months=1)

In [69]:
Brecknock['duration'] = Brecknock['period2'] - Brecknock['status_date']

In [70]:
Brecknock['duration'] = Brecknock['duration'].dt.days

In [71]:
Brecknock['study-start'] = '04/01/2010'

In [72]:
Brecknock['study-start'] = pd.to_datetime(Brecknock['study-start'], errors='coerce')

In [73]:
Brecknock['time-of-entry'] = Brecknock['status_date'] -  Brecknock['study-start']

In [74]:
Brecknock

,Unnamed: 0,Unnamed: 0.1,rates_code,rates_authority_id,rates_authority,region,location_code,address_no,address_street,address_town,...,V74,V75,V76,geometry,new_date,locaton_code,period2,duration,study-start,time-of-entry
396,31856,31856,00233002300007,E09000007,Camden,Inner London - West,11955063,23,BRECKNOCK ROAD,LONDON,...,13,CS,249.0,POINT (529735.7986901217 185054.3176096606),2010-04-01,11955063,2010-07-01,91,2010-04-01,0 days
397,31857,31857,00233002300007,E09000007,Camden,Inner London - West,11955063,23,BRECKNOCK ROAD,LONDON,...,13,CS,249.0,POINT (529735.7986901217 185054.3176096606),2010-04-01,11955063,2010-10-01,183,2010-04-01,0 days
398,31858,31858,00233002300007,E09000007,Camden,Inner London - West,11955063,23,BRECKNOCK ROAD,LONDON,...,13,CS,249.0,POINT (529735.7986901217 185054.3176096606),2010-04-01,11955063,2011-01-01,275,2010-04-01,0 days
399,31859,31859,00233002300007,E09000007,Camden,Inner London - West,11955063,23,BRECKNOCK ROAD,LONDON,...,13,CS,249.0,POINT (529735.7986901217 185054.3176096606),2010-04-01,11955063,2011-04-01,365,2010-04-01,0 days
400,31860,31860,00233002300007,E09000007,Camden,Inner London - West,11955063,23,BRECKNOCK ROAD,LONDON,...,13,CS,249.0,POINT (529735.7986901217 185054.3176096606),2010-04-01,11955063,2011-07-01,456,2010-04-01,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,401137,401137,00536000670001,E09000007,Camden,Inner London - West,11274243000,STUDIO 9 AT 6,CLIFF ROAD,LONDON,...,0,CO,203.0,POINT (529848.7992582474 184862.3169136087),2017-01-31 00:00:00,11274243000,2020-04-01,1156,2010-04-01,2497 days
2257,401138,401138,00536000670001,E09000007,Camden,Inner London - West,11274243000,STUDIO 9 AT 6,CLIFF ROAD,LONDON,...,0,CO,203.0,POINT (529848.7992582474 184862.3169136087),2017-01-31 00:00:00,11274243000,2020-07-01,1247,2010-04-01,2497 days
2258,401139,401139,00536000670001,E09000007,Camden,Inner London - West,11274243000,STUDIO 9 AT 6,CLIFF ROAD,LONDON,...,0,CO,203.0,POINT (529848.7992582474 184862.3169136087),2017-01-31 00:00:00,11274243000,2020-10-01,1339,2010-04-01,2497 days
2259,401140,401140,00536000670001,E09000007,Camden,Inner London - West,11274243000,STUDIO 9 AT 6,CLIFF ROAD,LONDON,...,0,CO,203.0,POINT (529848.7992582474 184862.3169136087),2017-01-31 00:00:00,11274243000,2021-01-01,1431,2010-04-01,2497 days


*** 

Set up columns to calculate the length of time a store is vacant 

In [75]:
Brecknock['time-at-open'] = 0

In [76]:
Brecknock['vacant'] = 0

In [49]:
#Brecknock = Brecknock.drop(Brecknock[(Brecknock['status'] == True) & (Brecknock['status_date'] == '2010-04-01')].index)

In [77]:
Brecknock = Brecknock.drop(Brecknock[(Brecknock['status'] == True)].index)

In [78]:
Brecknock

,Unnamed: 0,Unnamed: 0.1,rates_code,rates_authority_id,rates_authority,region,location_code,address_no,address_street,address_town,...,V76,geometry,new_date,locaton_code,period2,duration,study-start,time-of-entry,time-at-open,vacant
18,31434,31434,0023300030000B,E09000007,Camden,Inner London - West,11957063,3,BRECKNOCK ROAD,LONDON,...,249.0,POINT (529776.7972362216 185005.3245432741),2014-11-07 00:00:00,11957063,2015-01-01,55,2010-04-01,1681 days,0,0
19,31435,31435,0023300030000B,E09000007,Camden,Inner London - West,11957063,3,BRECKNOCK ROAD,LONDON,...,249.0,POINT (529776.7972362216 185005.3245432741),2014-11-07 00:00:00,11957063,2015-04-01,145,2010-04-01,1681 days,0,0
20,31436,31436,0023300030000B,E09000007,Camden,Inner London - West,11957063,3,BRECKNOCK ROAD,LONDON,...,249.0,POINT (529776.7972362216 185005.3245432741),2014-11-07 00:00:00,11957063,2015-07-01,236,2010-04-01,1681 days,0,0
21,31437,31437,0023300030000B,E09000007,Camden,Inner London - West,11957063,3,BRECKNOCK ROAD,LONDON,...,249.0,POINT (529776.7972362216 185005.3245432741),2014-11-07 00:00:00,11957063,2015-10-01,328,2010-04-01,1681 days,0,0
22,31438,31438,0023300030000B,E09000007,Camden,Inner London - West,11957063,3,BRECKNOCK ROAD,LONDON,...,249.0,POINT (529776.7972362216 185005.3245432741),2014-11-07 00:00:00,11957063,2016-01-01,420,2010-04-01,1681 days,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,364763,364763,00233003710035,E09000007,Camden,Inner London - West,9787634000,P J DENTAL SURGERY AT 37,BRECKNOCK ROAD,LONDON,...,437.0,POINT (529711.8009925119 185095.3217838815),2015-02-12 00:00:00,9787634000,2020-10-01,2058,2010-04-01,1778 days,0,0
2115,364764,364764,00233003710035,E09000007,Camden,Inner London - West,9787634000,P J DENTAL SURGERY AT 37,BRECKNOCK ROAD,LONDON,...,437.0,POINT (529711.8009925119 185095.3217838815),2015-02-12 00:00:00,9787634000,2021-01-01,2150,2010-04-01,1778 days,0,0
2116,364765,364765,00233003710035,E09000007,Camden,Inner London - West,9787634000,P J DENTAL SURGERY AT 37,BRECKNOCK ROAD,LONDON,...,437.0,POINT (529711.8009925119 185095.3217838815),2015-02-12 00:00:00,9787634000,2021-04-01,2240,2010-04-01,1778 days,0,0
2158,392475,392475,00536000500008,E09000007,Camden,Inner London - West,10992226000,8,CLIFF ROAD,LONDON,...,203.0,POINT (529847.7981505134 184861.3120667603),2016-08-22 00:00:00,10992226000,2016-10-01,40,2010-04-01,2335 days,0,0


In [79]:
Brecknock['location_code'] = Brecknock['location_code'].astype("category")

In [80]:
locations = list(Brecknock['location_code'].unique())

In [81]:
Brecknock = Brecknock.set_index('location_code')

In [82]:
Brecknock['survival'] = 0

In [83]:
Brecknock['survival'] = Brecknock['duration']

In [84]:
Brecknock['entry'] = Brecknock['time-of-entry'].dt.days

In [85]:
Brecknock['time-at-open'] = Brecknock['survival'] + Brecknock['entry']

In [86]:
Brecknock['stat'] = Brecknock['status'].astype(int)

In [87]:
Brecknock["StatusChange"] =  Brecknock['stat'].shift() != Brecknock['stat']

In [88]:
Brecknock["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [89]:
Brecknock["TenantChange"] =  Brecknock['name'].shift(-1) != Brecknock['name'] 

In [90]:
Brecknock["TenantChange"].iloc[0] = False

In [91]:
Brecknock

,Unnamed: 0,Unnamed: 0.1,rates_code,rates_authority_id,rates_authority,region,address_no,address_street,address_town,pc__pcs,...,duration,study-start,time-of-entry,time-at-open,vacant,survival,entry,stat,StatusChange,TenantChange
location_code,,,,,,,,,,,,,,,,,,,,,
11957063,31434,31434,0023300030000B,E09000007,Camden,Inner London - West,3,BRECKNOCK ROAD,LONDON,N7 0BL,...,55,2010-04-01,1681 days,1736,0,55,1681,0,False,False
11957063,31435,31435,0023300030000B,E09000007,Camden,Inner London - West,3,BRECKNOCK ROAD,LONDON,N7 0BL,...,145,2010-04-01,1681 days,1826,0,145,1681,0,False,False
11957063,31436,31436,0023300030000B,E09000007,Camden,Inner London - West,3,BRECKNOCK ROAD,LONDON,N7 0BL,...,236,2010-04-01,1681 days,1917,0,236,1681,0,False,False
11957063,31437,31437,0023300030000B,E09000007,Camden,Inner London - West,3,BRECKNOCK ROAD,LONDON,N7 0BL,...,328,2010-04-01,1681 days,2009,0,328,1681,0,False,False
11957063,31438,31438,0023300030000B,E09000007,Camden,Inner London - West,3,BRECKNOCK ROAD,LONDON,N7 0BL,...,420,2010-04-01,1681 days,2101,0,420,1681,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9787634000,364763,364763,00233003710035,E09000007,Camden,Inner London - West,P J DENTAL SURGERY AT 37,BRECKNOCK ROAD,LONDON,N7 0BT,...,2058,2010-04-01,1778 days,3836,0,2058,1778,0,False,False
9787634000,364764,364764,00233003710035,E09000007,Camden,Inner London - West,P J DENTAL SURGERY AT 37,BRECKNOCK ROAD,LONDON,N7 0BT,...,2150,2010-04-01,1778 days,3928,0,2150,1778,0,False,False
9787634000,364765,364765,00233003710035,E09000007,Camden,Inner London - West,P J DENTAL SURGERY AT 37,BRECKNOCK ROAD,LONDON,N7 0BT,...,2240,2010-04-01,1778 days,4018,0,2240,1778,0,False,True


In [93]:
counter = 0 
stoppingCondition = len(locations)
Brecknock2 = pd.DataFrame(data=None, columns=Brecknock.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Brecknock.loc[i, :].iloc[[ Brecknock.loc[i, :]['status_duration'].argmax()]]
        Brecknock2 = Brecknock2.append(m, ignore_index=False)
        counter += 1

In [94]:
Brecknock2 = Brecknock2.drop_duplicates()

In [95]:
Brecknock2['highstreet'] = 'Brecknock'

In [96]:
Brecknock2.to_csv('outputs_csv/Brecknock-vacant-survival.csv')